In [1]:
import pandas as pd 
import numpy as np
import re
import itertools
import spacy
import string
from spacy.lang.en import English
from spacy.lang.en.stop_words import STOP_WORDS
import en_core_web_sm
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline

In [2]:
df_data_raw = pd.read_csv('data/fake_job_postings.csv')

In [3]:
df_categorical = df_data_raw.select_dtypes(include = 'object').fillna('')
df_categorical.head()

,title,location,department,salary_range,company_profile,description,requirements,benefits,employment_type,required_experience,required_education,industry,function
0,Marketing Intern,"US, NY, New York",Marketing,,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,,Other,Internship,,,Marketing
1,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,Full-time,Not Applicable,,Marketing and Advertising,Customer Service
2,Commissioning Machinery Assistant (CMA),"US, IA, Wever",,,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,,,,,,
3,Account Executive - Washington DC,"US, DC, Washington",Sales,,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales
4,Bill Review Manager,"US, FL, Fort Worth",,,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider


In [4]:
str_bow_raw = ' '.join([' '.join(df_categorical[x].astype(str).tolist()) for x in df_categorical.columns])

In [58]:
def clean_text(str_text_raw):
#     translator = str.maketrans(string.punctuation, ' '*len(string.punctuation))
#     return(str_bow_raw.lower().translate(translator))
    return(str_bow_raw.lower())
str_bow = clean_text(str_bow_raw)

In [59]:
str_bow = str_bow[0:1000]

In [60]:
str_bow

'marketing intern customer service - cloud video production commissioning machinery assistant (cma) account executive - washington dc bill review manager accounting clerk head of content (m/f) lead guest service specialist    hp bsm sme customer service associate - part time  asp.net developer job opportunity at united states,new jersey talent sourcer (6 months fixed-term contract) applications developer, digital  installers account executive - sydney vp of sales - vault dragon hands-on qa leader  southend-on-sea traineeships under nas 16-18 year olds only visual designer process controls engineer - dcs plc ms office - pa marketing assistant front end developer    engagement manager  vice president, sales and sponsorship (businessfriend.com) customer service  h1b sponsor for l1/l2/opt marketing exec haad/dha licensed doctors opening in uae talent management process manager customer service associate  customer service technical specialist  software applications specialist craftsman asso

In [68]:
# nlp = English()
nlp = en_core_web_sm.load()
doc = nlp(str_bow)

In [69]:
l_token = [[token.text, token.lemma_, token.pos_] for token in doc if not token.is_punct | token.is_space | token.is_stop]
# l_token[1:100]

In [70]:
l_token

[['marketing', 'market', 'VERB'],
 ['intern', 'intern', 'NOUN'],
 ['customer', 'customer', 'NOUN'],
 ['service', 'service', 'NOUN'],
 ['cloud', 'cloud', 'NOUN'],
 ['video', 'video', 'NOUN'],
 ['production', 'production', 'NOUN'],
 ['commissioning', 'commissioning', 'NOUN'],
 ['machinery', 'machinery', 'NOUN'],
 ['assistant', 'assistant', 'NOUN'],
 ['cma', 'cma', 'PROPN'],
 ['account', 'account', 'NOUN'],
 ['executive', 'executive', 'PROPN'],
 ['washington', 'washington', 'PROPN'],
 ['dc', 'dc', 'PROPN'],
 ['bill', 'bill', 'PROPN'],
 ['review', 'review', 'PROPN'],
 ['manager', 'manager', 'PROPN'],
 ['accounting', 'accounting', 'PROPN'],
 ['clerk', 'clerk', 'PROPN'],
 ['head', 'head', 'NOUN'],
 ['content', 'content', 'NOUN'],
 ['m', 'm', 'PROPN'],
 ['f', 'f', 'PROPN'],
 ['lead', 'lead', 'NOUN'],
 ['guest', 'guest', 'NOUN'],
 ['service', 'service', 'NOUN'],
 ['specialist', 'specialist', 'NOUN'],
 ['hp', 'hp', 'PROPN'],
 ['bsm', 'bsm', 'PROPN'],
 ['sme', 'sme', 'PROPN'],
 ['customer', 'cus

In [74]:
pd_token = pd.DataFrame(l_token, columns=['Texto', 'Lemma', 'Part of Speech'])

In [75]:
pd_token.head()

,texto,lemma,Part of Speech
0,marketing,market,VERB
1,intern,intern,NOUN
2,customer,customer,NOUN
3,service,service,NOUN
4,cloud,cloud,NOUN
...,...,...,...
119,software,software,NOUN
120,applications,application,NOUN
121,specialist,specialist,PROPN
122,craftsman,craftsman,PROPN


In [9]:
tok_exp = nlp.tokenizer.explain(str_bow)

In [15]:
print(tok_exp)

[('TOKEN', 'marketing'), ('TOKEN', 'intern'), ('TOKEN', 'customer'), ('TOKEN', 'service'), ('TOKEN', '-'), ('TOKEN', 'cloud'), ('TOKEN', 'video'), ('TOKEN', 'production'), ('TOKEN', 'commissioning'), ('TOKEN', 'machinery'), ('TOKEN', 'assistant'), ('PREFIX', '('), ('TOKEN', 'cma'), ('SUFFIX', ')'), ('TOKEN', 'a')]
